In [133]:
from model_gens.gru_model_gen import GRUModel
from model_gens.lstm_model_gen import LSTMModel
from model_gens.meta_model_gen import MetaModel
import pandas as pd # type: ignore
import numpy as np # type: ignore
import tensorflow as tf # type: ignore
import logging

In [134]:
# Enable memory growth for GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

In [135]:
# Initialize models
market_predictor_lstm = LSTMModel()
market_predictor_gru = GRUModel()
market_predictor_meta = MetaModel()

In [136]:
# Load data paths
latest_data_path = "/home/amar/Documents/Projects/MarketPredictor/Datas/XAUUSD/latest_xauusd.csv"

In [137]:
# Load pre-trained models
market_predictor_lstm.load_model(model_name='models/lstm/XAUUSD_combined_lstm')
market_predictor_gru.load_model(model_name='models/gru/XAUUSD_combined_gru')
market_predictor_meta.load_meta_model(filename='models/meta/XAUUSD_combined_meta.pkl')

In [138]:
market_predictor_meta.lstm_model = market_predictor_lstm.model
market_predictor_meta.gru_model = market_predictor_gru.model

In [139]:
# Load scaler and feature shape
market_predictor_meta.load_scaler_and_shape(
    scaler_filename='/home/amar/Documents/Projects/MarketPredictor/models/scalers/meta/XAUUSD_combined_meta_scaler.pkl',
    shape_filename='/home/amar/Documents/Projects/MarketPredictor/models/shapes/meta/XAUUSD_combined_meta_shape.pkl'
)

In [140]:
market_predictor_gru.load_scaler(
    scaler_filename='/home/amar/Documents/Projects/MarketPredictor/models/scalers/gru/XAUUSD_gru_scaler.pkl'
)
market_predictor_lstm.load_scaler(
    scaler_filename='/home/amar/Documents/Projects/MarketPredictor/models/scalers/lstm/XAUUSD_lstm_scaler.pkl'
)

In [141]:
# Prepare latest data
latest_data = pd.read_csv(latest_data_path, parse_dates=['Date Time'], index_col='Date Time')
preprocessed_latest_data = market_predictor_lstm.preprocess_data(latest_data)

In [142]:
# Ensure latest data has enough time steps
if preprocessed_latest_data.shape[0] < 60:
    raise ValueError("Not enough data after preprocessing to create sequences.")

In [143]:
# Generate meta-features for the latest data
# Assuming the scaler has already been fitted during training
# from sklearn.preprocessing import MinMaxScaler
# market_predictor_meta.scaler = MinMaxScaler(feature_range=(0, 1))
# scaled_data = market_predictor_meta.scaler.transform(preprocessed_latest_data)

In [144]:
preprocessed_latest_data[-60:]

,Open,High,Low,Close,Change,Volume,MA_20,MA_50,RSI,MACD,...,ATR,Stochastic,Donchian_upper,Donchian_lower,Fib_0,Fib_0.236,Fib_0.382,Fib_0.5,Fib_0.618,Fib_1
Date Time,,,,,,,,,,,,,,,,,,,,,
2024-05-02 06:35:00,2299.35,2300.03,2299.35,2300.03,0.69,13,2298.7565,2303.1382,47.295817,-0.789555,...,0.720426,59.891107,2301.92,2296.29,2296.29,2301.41828,2304.59086,2307.155,2309.71914,2318.02
2024-05-02 06:40:00,2300.01,2300.01,2299.44,2299.44,-0.59,20,2298.6680,2302.9062,44.271860,-0.744204,...,0.711085,60.133091,2301.80,2296.29,2296.29,2301.35928,2304.49536,2307.030,2309.56464,2317.77
2024-05-02 06:45:00,2299.41,2299.47,2299.07,2299.07,-0.37,14,2298.6650,2302.6648,42.439310,-0.729707,...,0.688810,57.747572,2301.80,2296.29,2296.29,2301.02888,2303.96056,2306.330,2308.69944,2316.37
2024-05-02 06:50:00,2299.06,2299.06,2298.46,2298.49,-0.58,18,2298.6970,2302.4130,39.667439,-0.756301,...,0.683168,46.476624,2301.80,2296.29,2296.29,2300.71264,2303.44868,2305.660,2307.87132,2315.03
2024-05-02 06:55:00,2298.50,2298.85,2298.49,2298.85,0.36,22,2298.7590,2302.1770,42.191254,-0.739800,...,0.660035,41.165383,2301.80,2296.29,2296.29,2300.64892,2303.34554,2305.525,2307.70446,2314.76
2024-05-02 07:00:00,2298.87,2298.87,2298.31,2298.37,-0.48,15,2298.8015,2301.9378,39.800589,-0.756732,...,0.652875,31.080463,2301.80,2296.29,2296.29,2300.56868,2303.21566,2305.355,2307.49434,2314.42
2024-05-02 07:05:00,2298.41,2298.57,2298.39,2298.54,0.17,19,2298.8610,2301.7210,41.074077,-0.747812,...,0.620468,21.922634,2301.80,2296.29,2296.29,2300.56868,2303.21566,2305.355,2307.49434,2314.42
2024-05-02 07:10:00,2298.57,2299.48,2298.57,2299.48,0.94,15,2299.0130,2301.5146,47.666515,-0.657316,...,0.643330,19.352399,2301.80,2296.29,2296.29,2300.42708,2302.98646,2305.055,2307.12354,2313.82
2024-05-02 07:15:00,2299.61,2300.55,2299.61,2300.55,1.07,17,2299.2150,2301.3234,53.978181,-0.493568,...,0.673855,34.765998,2301.80,2296.53,2296.29,2299.92912,2302.18044,2304.000,2305.81956,2311.71


In [145]:
data = market_predictor_lstm.scaler.fit_transform(preprocessed_latest_data)

In [146]:
initial_data = data[-60:]

In [147]:
initial_data = initial_data[np.newaxis, :, :]

In [148]:
initial_data.shape

(1, 60, 24)

In [149]:
meta_features = market_predictor_meta.generate_meta_features(
    lstm_model=market_predictor_lstm.model, 
    gru_model=market_predictor_gru.model, 
    X=initial_data
)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step


In [150]:
decoded = market_predictor_meta.output_decoder(
    predictions=meta_features[0], scaler=market_predictor_lstm.scaler, data=preprocessed_latest_data)

In [151]:
decoded

array([2298.93255823, 2288.06076274])

In [152]:
meta_pred = market_predictor_meta.meta_model.predict(meta_features)

In [153]:
dec = market_predictor_meta.output_decoder(
    predictions=meta_pred, scaler=market_predictor_lstm.scaler, data=preprocessed_latest_data)

In [154]:
dec

array([2295.87679524])